# Next edit prediction
## Introduction
- IDE's have tab completion to suggest the next snippet code. 
- We've also seen how we can refactor code with a prompt.
- Now we combine both : when we change some of the code, what edits are next ?

This feature was initally introduced by Cursor as Cursor prediction:
- https://docs.cursor.com/tab/advanced-features#cursor-prediction

Soon other vendors implemented the same feature:
- https://www.augmentcode.com/blog/introducing-next-edit-for-vscode
- https://docs.augmentcode.com/using-augment/next-edit
- https://www.augmentcode.com/blog/the-ai-research-behind-next-edit

- https://code.visualstudio.com/blogs/2025/02/12/next-edit-suggestions
- https://githubnext.com/projects/copilot-next-edit-suggestions/
- https://www.copilotcandothat.com/tips/nes-refactor

Thanks to Zed there is now an Open Source model called `Zeta`
- https://zed.dev/blog/edit-prediction
- https://www.youtube.com/watch?v=r1A268kA1uM
- https://zed.dev/edit-prediction
- https://huggingface.co/zed-industries/zeta

More information can be found at:
- https://huggingface.co/zed-industries/zeta/discussions/5
- https://www.youtube.com/watch?v=3Tcf_sms0gk
- https://www.youtube.com/watch?v=51u9o4rnV1E
- https://www.youtube.com/watch?v=r1A268kA1uM
- https://huggingface.co/bartowski/zed-industries_zeta-GGUF

## Installation

In [1]:
%pip install -Uq transformers torch accelerate peft huggingface_hub

Note: you may need to restart the kernel to use updated packages.


## Use GPU if on Mac

In [2]:
import torch
device = "cpu"
if torch.backends.mps.is_available():
    device = "mps"

## Download model
- We're using the Zeta model <https://huggingface.co/datasets/zed-industries/zeta>

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("zed-industries/zeta")
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained("zed-industries/zeta")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## The instruction prompt
- Using special tokens `<|editable_region_start|>` , `<|editable_region_end|>`, `|user_cursor_is_here|>` in the prompt, we indicate where we want edits, and where our cursor is currently located.

In the example the edit made was adding `.capitalize` to the property `name`:
```
- @name  = name
+ @name  = name.capitalize
```

Now we craft a next edit suggestion using this prompt template:

In [4]:
input_prompt = """### Instruction:
You are a code completion assistant and your task is to analyze user edits and then rewrite an excerpt that the user provides, suggesting the appropriate edits within the excerpt, taking into account the cursor location.

### User Edits:

User edited "models/customer.rb":

```diff
@@ -2,5 +2,5 @@
class Customer
def initialize
- @name  = name
+ @name  = name.capitalize
@email  = email
@phone  = phone
```

### User Excerpt:
def initialize
<|editable_region_start|>
@name  = name.capitalize<|user_cursor_is_here|>
@email  = email
@phone  = phone
@address  = address
end

def to_s
@name 
end

<|editable_region_end|>
private

def validate_email
@email .include?('@')

### Response:
"""


## Run the model with the prompt

In [5]:
inputs = tokenizer(input_prompt, return_tensors="pt").to(device)
# outputs = model.generate(**inputs, max_length=128)
outputs = model.generate(**inputs)

next_edit = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(input_prompt):]
print(next_edit)

<|editable_region_start|>
@name  = name.capitalize
@email  = email.capitalize
@phone  = phone.capitalize
@address  = address.capitalize
end

def to_s
@name 
end

<|editable_region_end|>
private

def validate_email
@email .include?('@')

### Response:
<|editable_region_start|>
@name  = name.capitalize
@email  = email.capitalize
@phone  = phone.capitalize
@address  = address.capitalize
end

def to_s
@name 
end

<|editable_region_end|>
private

def validate_email
@email .include?('@')

### Response:
<|editable_region_start|>
@name  = name.capitalize
@email  = email.capitalize
@phone  = phone.capitalize
@address  = address.capitalize
end

def to_s
@name 
end

<|editable_region_end|>
private

def validate_email
@email .include?('@')

### Response:
<|editable_region_start|>
@name  = name.capitalize
@email  = email.capitalize
@phone  = phone.capitalize
@address  = address.capitalize
end

def to_s
@name 
end

<|editable_region_end|>
private

def validate_email
@email .include?('@')

### Respon

## A diff helper

In [6]:
%pip install -q colorama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [7]:
from colorama import Fore, Back, Style
import difflib

def diff_code(a: str, b: str) -> str:
    output = []
    matcher = difflib.SequenceMatcher(None, a, b)

    green = Back.GREEN
    red = Back.RED
    endgreen = Back.WHITE
    endred = Back.WHITE

    output.append(Back.WHITE + Fore.BLACK)
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == 'equal':
            output.append(a[a0:a1])
        elif opcode == 'insert':
            output.append(f'{green}{b[b0:b1]}{endgreen}')
        elif opcode == 'delete':
            output.append(f'{red}{a[a0:a1]}{endred}')
        elif opcode == 'replace':
            output.append(f'{green}{b[b0:b1]}{endgreen}')
            output.append(f'{red}{a[a0:a1]}{endred}')
    output.append(Style.RESET_ALL)
    return ''.join(output)

## Extract the orginal code block

In [8]:
orginal_code = input_prompt.split("<|editable_region_start|>")[1].split("<|editable_region_end|>")[0]
print("Original code:")
print(orginal_code)

Original code:

@name  = name.capitalize<|user_cursor_is_here|>
@email  = email
@phone  = phone
@address  = address
end

def to_s
@name 
end




## Extract the new suggested code

In [9]:
updated_code = next_edit.split("<|editable_region_start|>")[1].split("<|editable_region_end|>")[0]
print(updated_code)


@name  = name.capitalize
@email  = email.capitalize
@phone  = phone.capitalize
@address  = address.capitalize
end

def to_s
@name 
end




## The results
We can see the suggested next edits: add the capatalize to the other properties too.



In [10]:
print(diff_code(orginal_code, updated_code))


@name  = name.capitalize<|user_cursor_is_here|>
@email  = email.capitalize
@phone  = phone.capitalize
@address  = address.capitalize
end

def to_s
@name 
end


